# Dynamic Padding :- 
pad the sentences at the batch creation, to the length of the longest sentences , and this technique is called dynamic paddingpad the sentences at the batch creation, to the length of the longest sentences , and this technique is called dynamic padding

* Advantages
 :- All the batches will have the smallest size possible
* disadvantage
  :-
  Dynamic shapes don't work well on all accelerators

In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer

In [5]:
raw_dataset = load_dataset('glue',"mrpc")
checkpoint = 'bert-base-cased'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [6]:
#making a tokenzer function which tokenize the given sentences
def tokenize_function(ex):
    return tokenizer(
        ex['sentence1'],ex['sentence2'],padding="max_length",truncation=True,max_length=128
    )


In [7]:
#applying the tokenizer functin
tokenized_dataset = raw_dataset.map(tokenize_function,batched=True)

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [8]:
#performing preprocessing on tokenized dataset
tokenized_dataset = tokenized_dataset.remove_columns(["idx","sentence1","sentence2"])
tokenized_dataset = tokenized_dataset.rename_column("label","labels")
tokenized_dataset = tokenized_dataset.with_format("torch")

In [12]:
# we can then use our dataset in a standard PYtorch DataLoader.
#As expected we get batches of fixed shapes
from torch.utils.data import DataLoader

In [10]:
train_dataloader = DataLoader(tokenized_dataset["train"],batch_size=16,shuffle=True)


In [11]:
for step,batch in enumerate(train_dataloader):
    print(batch['input_ids'].shape)
    if step>5:
        break

torch.Size([16, 128])
torch.Size([16, 128])
torch.Size([16, 128])
torch.Size([16, 128])
torch.Size([16, 128])
torch.Size([16, 128])
torch.Size([16, 128])


# To apply dynamic padding we postpone the padding in the preprocessing function


In [13]:
from datasets import load_dataset
from transformers import AutoTokenizer

In [14]:
raw_dataset = load_dataset('glue',"mrpc")
checkpoint = 'bert-base-cased'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [18]:
#making a tokenzer function which tokenize the given sentences
# here we remove the padding stufs as we dicucussed
def tokenize_function(ex):
    return tokenizer(
        ex['sentence1'],ex['sentence2'],truncation=True
    )


In [16]:
#applying the tokenizer functin
tokenized_dataset = raw_dataset.map(tokenize_function,batched=True)

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [17]:
#performing preprocessing on tokenized dataset
tokenized_dataset = tokenized_dataset.remove_columns(["idx","sentence1","sentence2"])
tokenized_dataset = tokenized_dataset.rename_column("label","labels")
tokenized_dataset = tokenized_dataset.with_format("torch")

In [19]:
#each batch then has a different size, but there is no needless padding

In [20]:
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding

In [21]:
data_collator = DataCollatorWithPadding(tokenizer)
train_dataloader = DataLoader(
    tokenized_dataset['train'],batch_size=16,shuffle=True,collate_fn=data_collator
)

In [22]:
# now we each batch shape is diffenet due to appication on dynamic padding
for step,batch in enumerate(train_dataloader):
    print(batch['input_ids'].shape)
    if step>5:
        break

torch.Size([16, 74])
torch.Size([16, 63])
torch.Size([16, 81])
torch.Size([16, 86])
torch.Size([16, 93])
torch.Size([16, 86])
torch.Size([16, 88])
